# BERT-language2label

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Batcher ModelSupport Datasets TextModels

import Datasets
import Foundation
import ModelSupport
import TensorFlow
import TextModels
import PythonKit

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Batcher
		ModelSupport
		Datasets
		TextModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpbbeal5d9/swift-install
[1/5] Compiling STBImage stb_image_write.c
[2/5] Compiling Batcher Backend.swift
[3/5] Compiling STBImage stb_image.c
[4/5] Compiling SwiftProtobuf AnyMessageStorage.swift
/notebooks/language2motion.gt/swift-install/package/.build/checkouts/swift-protobuf/Sources/SwiftProtobuf/BinaryDelimited.swift:198:7: warning: variable 'readBuffer' was never mutated; consider changing to 'let' constant
  var readBuffer = UnsafeMutablePointer<UInt8>.allocate(capacity: 1)
  ~~~ ^
  let
[5/6] Compiling ModelSupport BijectiveDictionary.swift
[6/7] Compiling Datasets BostonHousing.swift
[7/8] Compiling TextModels Attention.swift
[8/9] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[9/9] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
let bertPretrained = BERT.PreTrainedModel.bertBase(cased: false, multilingual: false)
let workspaceURL = URL(
    fileURLWithPath: "bert_models", isDirectory: true,
    relativeTo: URL(
        fileURLWithPath: NSTemporaryDirectory(),
        isDirectory: true))
let bert = try BERT.PreTrainedModel.load(bertPretrained)(from: workspaceURL)
var bertClassifier = BERTClassifier(bert: bert, classCount: 1)

Loading BERT pre-trained model 'BERT Base Uncased'.
Loading resource: uncased_L-12_H-768_A-12


# load dataset

In [ ]:
// TODO: 
// + load csv
// + split into train/dev
// + convert to [Example]
// + get sorted labels
// - integrate with Language2Label

In [12]:
import PythonKit

In [13]:
let pd  = Python.import("pandas")
let model_selection  = Python.import("sklearn.model_selection")

In [14]:
let dsURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/labels.csv")

In [17]:
let df = pd.read_csv(dsURL.path)
df.head()

   sample_id  ...                      label
0          1  ...         Walking or running
1          2  ...  Walking forward few steps
2          3  ...         Walking or running
3          4  ...         Walking or running
4          5  ...  Walking forward few steps

[5 rows x 3 columns]


In [85]:
let labels: [String] = df.label.unique().sorted().map {String($0)!}
print(type(of:labels))
print(type(of:labels[0]))

Array<String>
String


In [20]:
public struct Example {
    public let id: String
    public let text: String
    public let label: String?

    public init(id: String, text: String, label: String?) {
        self.id = id
        self.text = text
        self.label = label
    }
}

In [69]:
let (train_df, test_df) = model_selection.train_test_split(df, test_size: 0.2).tuple2
train_df

      sample_id  ...                      label
1592       1621  ...         Walking or running
1699       1728  ...  Walking forward few steps
2555       3071  ...         Walking or running
2082       2111  ...            Doing something
2402       2758  ...            Doing something
...         ...  ...                        ...
2446       2867  ...         Walking or running
1102       1110  ...            Doing something
2678       3304  ...         Walking or running
1477       1485  ...         Walking or running
717         724  ...  Walking forward few steps

[2409 rows x 3 columns]


In [65]:
func Df2Example(df: PythonObject) -> [Example] {
    return Python.list(df.iterrows()).map {
        (rowObj: PythonObject) -> Example in 
        let row = rowObj.tuple2.1
        let sample_id: String = "\(row.sample_id)" // Int to String
        let text: String = String(row.text)!
        let label: String? = String(row.label)
        return Example(id: sample_id, text: text, label: label)
    }
}

In [71]:
let trainExamples = Df2Example(df: train_df.iloc[0..<3])
let testExamples = Df2Example(df: test_df.iloc[0..<3])
(trainExamples, testExamples)

▿ 2 elements
  ▿ .0 : 3 elements
    ▿ 0 : Example
      - id : "1621"
      - text : "A person is walking carefully avoiding obstacles."
      ▿ label : Optional<String>
        - some : "Walking or running"
    ▿ 1 : Example
      - id : "1728"
      - text : "A person stumbles slightly while walking backwards but quickly recovers and continues moving backwards."
      ▿ label : Optional<String>
        - some : "Walking forward few steps"
    ▿ 2 : Example
      - id : "3071"
      - text : "A human is hitting something"
      ▿ label : Optional<String>
        - some : "Walking or running"
  ▿ .1 : 3 elements
    ▿ 0 : Example
      - id : "1109"
      - text : "Subject starts a halfhearted jog."
      ▿ label : Optional<String>
        - some : "Doing something"
    ▿ 1 : Example
      - id : "1992"
      - text : "A person walks forwards, turns around a walks back."
      ▿ label : Optional<String>
        - some : "Walking forward few steps"
    ▿ 2 : Example
      - id : "1557"

In [3]:
public struct Language2Label {
    public let directoryURL: URL
    public let trainExamples: [Example]
    public let devExamples: [Example]
    public let testExamples: [Example]
    public let maxSequenceLength: Int
    public let batchSize: Int
    // TODO: add labels

    public typealias ExampleIterator = IndexingIterator<[Example]>
    public typealias TrainDataIterator = PrefetchIterator<
        GroupedIterator<MapIterator<ExampleIterator, DataBatch>>
    >
    public typealias DevDataIterator = GroupedIterator<MapIterator<ExampleIterator, DataBatch>>
    public typealias TestDataIterator = DevDataIterator

    public var trainDataIterator: TrainDataIterator
    public var devDataIterator: DevDataIterator
    public var testDataIterator: TestDataIterator
}

//===-----------------------------------------------------------------------------------------===//
// Data
//===-----------------------------------------------------------------------------------------===//

extension Language2Label {
    /// Language2Label example.
    public struct Example {
        public let id: String
        public let sentence: String // change to annotation
        public let isAcceptable: Bool? // TODO: change to 5 labels

        public init(id: String, sentence: String, isAcceptable: Bool?) {
            self.id = id
            self.sentence = sentence
            self.isAcceptable = isAcceptable
        }
    }

    /// Language2Label data batch.
    public struct DataBatch: KeyPathIterable {
        public var inputs: TextBatch  // TODO: !!! Mutable in order to allow for batching.
        public var labels: Tensor<Int32>?  // TODO: !!! Mutable in order to allow for batching.

        public init(inputs: TextBatch, labels: Tensor<Int32>?) {
            self.inputs = inputs
            self.labels = labels
        }
    }

    /// URL pointing to the downloadable ZIP file that contains the CoLA dataset.
    private static let url: URL = URL(
        string: String(
            "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/"
                + "o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4"))!

    internal enum FileType: String {
        case train = "train"
        case dev = "dev"
        case test = "test" // TODO: kill
    }
    // TODO: load csv file, split into train/test, extract row annotation -> sentence, label -> label
    internal static func load(fromFile fileURL: URL, fileType: FileType) throws -> [Example] {
        let lines = try parse(tsvFileAt: fileURL)

        if fileType == .test {
            // The test data file has a header.
            return lines.dropFirst().enumerated().map { (i, lineParts) in
                Example(id: lineParts[0], sentence: lineParts[1], isAcceptable: nil)
            }
        }

        return lines.enumerated().map { (i, lineParts) in
            Example(id: lineParts[0], sentence: lineParts[3], isAcceptable: lineParts[1] == "1")
        }
    }
}

internal func parse(tsvFileAt fileURL: URL) throws -> [[String]] {
    try Data(contentsOf: fileURL).withUnsafeBytes {
        $0.split(separator: UInt8(ascii: "\n")).map {
            $0.split(separator: UInt8(ascii: "\t"), omittingEmptySubsequences: false)
                .map { String(decoding: UnsafeRawBufferPointer(rebasing: $0), as: UTF8.self) }
        }
    }
}

extension Language2Label {
    public init(
        exampleMap: @escaping (Example) -> DataBatch,
        taskDirectoryURL: URL,
        maxSequenceLength: Int,
        batchSize: Int,
        dropRemainder: Bool
    ) throws {
        self.directoryURL = taskDirectoryURL.appendingPathComponent("CoLA")
        let dataURL = directoryURL.appendingPathComponent("data")
        let compressedDataURL = dataURL.appendingPathComponent("downloaded-data.zip")

        // Download the data, if necessary.
        try download(from: Language2Label.url, to: compressedDataURL)

        // Extract the data, if necessary.
        let extractedDirectoryURL = compressedDataURL.deletingPathExtension()
        if !FileManager.default.fileExists(atPath: extractedDirectoryURL.path) {
            try extract(zipFileAt: compressedDataURL, to: extractedDirectoryURL)
        }

        #if false
            // FIXME: Need to generalize `DatasetUtilities.downloadResource` to accept
            // arbitrary full URLs instead of constructing full URL from filename and
            // file extension.
            DatasetUtilities.downloadResource(
                filename: "\(subDirectory)", fileExtension: "zip",
                remoteRoot: url.deletingLastPathComponent(),
                localStorageDirectory: directory)
        #endif

        // Load the data files into arrays of examples.
        let dataFilesURL = extractedDirectoryURL.appendingPathComponent("CoLA")
        self.trainExamples = try Language2Label.load(
            fromFile: dataFilesURL.appendingPathComponent("train.tsv"),
            fileType: .train)
        self.devExamples = try Language2Label.load(
            fromFile: dataFilesURL.appendingPathComponent("dev.tsv"),
            fileType: .dev)
        self.testExamples = try Language2Label.load(
            fromFile: dataFilesURL.appendingPathComponent("test.tsv"),
            fileType: .test)

        self.maxSequenceLength = maxSequenceLength
        self.batchSize = batchSize

        // Create the data iterators used for training and evaluating.
        self.trainDataIterator = trainExamples.shuffled().makeIterator()  // TODO: [RNG] Seed support.
            .map(exampleMap)
            .grouped(
                keyFn: { _ in 0 },
                sizeFn: { _ in batchSize / maxSequenceLength },
                reduceFn: {
                    DataBatch(
                        inputs: padAndBatch(
                            textBatches: $0.map { $0.inputs }, maxLength: maxSequenceLength),
                        labels: Tensor.batch($0.map { $0.labels! }))
                },
                dropRemainder: dropRemainder
            )
            .prefetched(count: 2)
        self.devDataIterator = devExamples.makeIterator()
            .map(exampleMap)
            .grouped(
                keyFn: { _ in 0 },
                sizeFn: { _ in batchSize / maxSequenceLength },
                reduceFn: {
                    DataBatch(
                        inputs: padAndBatch(
                            textBatches: $0.map { $0.inputs }, maxLength: maxSequenceLength),
                        labels: Tensor.batch($0.map { $0.labels! }))
                },
                dropRemainder: dropRemainder
            )
        self.testDataIterator = testExamples.makeIterator()
            .map(exampleMap)
            .grouped(
                keyFn: { _ in 0 },
                sizeFn: { _ in batchSize / maxSequenceLength },
                reduceFn: {
                    DataBatch(
                        inputs: padAndBatch(
                            textBatches: $0.map { $0.inputs }, maxLength: maxSequenceLength),
                        labels: nil)
                },
                dropRemainder: dropRemainder
            )
    }
}

In [4]:
// Regarding the batch size, note that the way batching is performed currently is that we bucket
// input sequences based on their length (e.g., first bucket contains sequences of length 1 to 10,
// second 11 to 20, etc.). We then keep processing examples in the input data pipeline until a
// bucket contains enough sequences to form a batch. The batch size specified in the task
// constructor specifies the *total number of tokens in the batch* and not the total number of
// sequences. So, if the batch size is set to 1024, the first bucket (i.e., lengths 1 to 10)
// will need 1024 / 10 = 102 examples to form a batch (every sentence in the bucket is padded
// to the max length of the bucket). This kind of bucketing is common practice with NLP models and
// it is done to improve memory usage and computational efficiency when dealing with sequences of
// varied lengths. Note that this is not used in the original BERT implementation released by
// Google and so the batch size setting here is expected to differ from that one.
let maxSequenceLength = 128
let batchSize = 1024

// Create a function that converts examples to data batches.
let exampleMapFn: (Language2Label.Example) -> Language2Label.DataBatch = { example -> Language2Label.DataBatch in
    let textBatch = bertClassifier.bert.preprocess(
        sequences: [example.sentence],
        maxSequenceLength: maxSequenceLength)
    return Language2Label.DataBatch( // TODO: get label idx
        inputs: textBatch, labels: example.isAcceptable.map { Tensor($0 ? 1 : 0) })
}

var dataset = try Language2Label(
    exampleMap: exampleMapFn,
    taskDirectoryURL: workspaceURL,
    maxSequenceLength: maxSequenceLength,
    batchSize: batchSize,
    dropRemainder: true)

print("Dataset acquired.")

Dataset acquired.


In [5]:
dataset.directoryURL

▿ bert_models/CoLA/ -- file:///tmp/
  ▿ _storage : Optional<Storage>
    ▿ some : Storage
      ▿ boxedRequiresCopyOnWrite : <URLBox: 0x366ae80>


In [6]:
dataset.trainExamples.count

8551


In [7]:
dataset.trainExamples[0]

▿ Example
  - id : "gj04"
  - sentence : "Our friends won\'t buy this analysis, let alone the next one we propose."
  ▿ isAcceptable : Optional<Bool>
    - some : true


In [8]:
dataset.testExamples[0]

▿ Example
  - id : "0"
  - sentence : "Bill whistled past the house."
  - isAcceptable : nil


In [9]:
dataset.devExamples[0]

▿ Example
  - id : "gj04"
  - sentence : "The sailors rode the breeze clear of the rocks."
  ▿ isAcceptable : Optional<Bool>
    - some : true


In [10]:
var optimizer = WeightDecayedAdam(
    for: bertClassifier,
    learningRate: LinearlyDecayedParameter(
        baseParameter: LinearlyWarmedUpParameter(
            baseParameter: FixedParameter<Float>(2e-5),
            warmUpStepCount: 10,
            warmUpOffset: 0),
        slope: -5e-7,  // The LR decays linearly to zero in 100 steps.
        startStep: 10),
    weightDecayRate: 0.01,
    maxGradientGlobalNorm: 1)

print("Training BERT for the Language2Label task!")
for epoch in 1...3 {
    print("[Epoch \(epoch)]")
    Context.local.learningPhase = .training
    var trainingLossSum: Float = 0
    var trainingBatchCount = 0
    var trainingDataIterator = dataset.trainDataIterator

    while let batch = withDevice(.cpu, perform: { trainingDataIterator.next() }) {
        let (documents, labels) = (batch.inputs, Tensor<Float>(batch.labels!))
        let (loss, gradients) = valueWithGradient(at: bertClassifier) { model -> Tensor<Float> in
            let logits = model(documents)
            return sigmoidCrossEntropy(
                logits: logits.squeezingShape(at: -1),
                labels: labels,
                reduction: { $0.mean() })
        }

        trainingLossSum += loss.scalarized()
        trainingBatchCount += 1
        optimizer.update(&bertClassifier, along: gradients)

        print(
            """
              Training loss: \(trainingLossSum / Float(trainingBatchCount))
            """
        )
    }

    Context.local.learningPhase = .inference
    var devLossSum: Float = 0
    var devBatchCount = 0
    var devDataIterator = dataset.devDataIterator
    var devPredictedLabels = [Bool]()
    var devGroundTruth = [Bool]()
    while let batch = withDevice(.cpu, perform: { devDataIterator.next() }) {
        let (documents, labels) = (batch.inputs, batch.labels!)
        let logits = bertClassifier(documents)
        let loss = sigmoidCrossEntropy(
            logits: logits.squeezingShape(at: -1),
            labels: Tensor<Float>(labels),
            reduction: { $0.mean() }
        )
        devLossSum += loss.scalarized()
        devBatchCount += 1

        let predictedLabels = sigmoid(logits.squeezingShape(at: -1)) .>= 0.5
        devPredictedLabels.append(contentsOf: predictedLabels.scalars)
        devGroundTruth.append(contentsOf: labels.scalars.map { $0 == 1 })
    }

    let mcc = matthewsCorrelationCoefficient(
        predictions: devPredictedLabels,
        groundTruth: devGroundTruth)

    print(
        """
          MCC: \(mcc)
          Eval loss: \(devLossSum / Float(devBatchCount))
        """
    )
}

Training BERT for the Language2Label task!
[Epoch 1]
  Training loss: 1.0054466


: 